In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

--- One-Class SVM ---

In [8]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix, classification_report, precision_score, accuracy_score, roc_auc_score

ocsvc= OneClassSVM(gamma= 'auto', nu= 0.01)
ocsvc.fit(X_train[y_train== 0])
y_pred_ocsvc= ocsvc.predict(X_test)
y_pred_ocsvc= [1 if x== -1 else 0 for x in y_pred_ocsvc]
y_pred_prob_ocsvc= ocsvc.decision_function(X_test)

cm= confusion_matrix(y_test, y_pred_ocsvc)
print(classification_report(y_test, y_pred_ocsvc, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_ocsvc, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_ocsvc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ocsvc))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      8517
           1       0.22      0.05      0.08       912

    accuracy                           0.89      9429
   macro avg       0.56      0.51      0.51      9429
weighted avg       0.84      0.89      0.86      9429

Precision: 0.21782178217821782
Accuracy: 0.8911867642379891
AUC: 0.3958033365673195


--- Combining One-Class SVM with Traditional Classifiers ---

In [9]:
ocsvc= OneClassSVM(gamma='auto', nu=0.01)
ocsvc.fit(X_train[y_train== 0])

X_train_ocsvc_scores= ocsvc.decision_function(X_train).reshape(-1, 1)
X_test_ocsvc_scores= ocsvc.decision_function(X_test).reshape(-1, 1)
X_train_combined= np.hstack((X_train, X_train_ocsvc_scores))
X_test_combined= np.hstack((X_test, X_test_ocsvc_scores))

--- LogisticRegression ---

In [10]:
from sklearn.linear_model import LogisticRegression
lr_combined= LogisticRegression(max_iter= 1000, random_state= 15)
lr_combined.fit(X_train_combined, y_train)
y_pred_lr_combined = lr_combined.predict(X_test_combined)
y_pred_prob_lr_combined= lr_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_combined, labels= lr_combined.classes_)
print(classification_report(y_test, y_pred_lr_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr_combined))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.56      0.13      0.21       912

    accuracy                           0.91      9429
   macro avg       0.74      0.56      0.58      9429
weighted avg       0.88      0.91      0.88      9429

Precision: 0.5601851851851852
Accuracy: 0.9060345741860218
AUC: 0.8541337088464969


--- DecisionTreeClassifier ---

In [11]:
from sklearn.tree import DecisionTreeClassifier
dt_combined = DecisionTreeClassifier(random_state=15)
dt_combined.fit(X_train_combined, y_train)
y_pred_dt_combined = dt_combined.predict(X_test_combined)
y_pred_prob_dt_combined= dt_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_combined, labels= dt_combined.classes_)
print(classification_report(y_test, y_pred_dt_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_combined))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      8517
           1       0.24      0.27      0.25       912

    accuracy                           0.85      9429
   macro avg       0.58      0.59      0.58      9429
weighted avg       0.85      0.85      0.85      9429

Precision: 0.23902439024390243
Accuracy: 0.8465372786085481
AUC: 0.5881687347698823


--- Tuned - DecisionTreeClassifier ---

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_combined= GridSearchCV(estimator= dt_combined, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_combined.fit(X_train_combined, y_train)

print("Best Parameters:", gs_dt_combined.best_params_)
print("Best Precision Score:", gs_dt_combined.best_score_)

Best Parameters: {'criterion': 'gini', 'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 10, 'min_samples_split': 2}
Best Precision Score: 0.5083388503993207


In [13]:
tuned_dt_combined= gs_dt_combined.best_estimator_
y_pred_tuned_dt_combined= tuned_dt_combined.predict(X_test_combined)
y_pred_prob_tuned_dt_combined= tuned_dt_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_combined, labels= tuned_dt_combined.classes_)
print(classification_report(y_test, y_pred_tuned_dt_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_combined))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.43      0.07      0.12       912

    accuracy                           0.90      9429
   macro avg       0.67      0.53      0.53      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.4305555555555556
Accuracy: 0.9011560080602397
AUC: 0.8020015824903342


--- KNeighborsClassifier ---

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn_combined= KNeighborsClassifier()
knn_combined.fit(X_train_combined, y_train)
y_pred_knn_combined= knn_combined.predict(X_test_combined)
y_pred_prob_knn_combined= knn_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_combined, labels= knn_combined.classes_)
print(classification_report(y_test, y_pred_knn_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_combined))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      8517
           1       0.36      0.11      0.17       912

    accuracy                           0.90      9429
   macro avg       0.64      0.55      0.56      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.36428571428571427
Accuracy: 0.8952168840810266
AUC: 0.7026886629218343


--- Tuned - KNeighborsClassifier ---

In [15]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_combined= GridSearchCV(estimator= knn_combined, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_combined.fit(X_train_combined, y_train)
print("Best Parameters:", gs_knn_combined.best_params_)
print("Best Precision Score:", gs_knn_combined.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 38, 'weights': 'uniform'}
Best Precision Score: 0.7323485838779956


In [16]:
tuned_knn_combined= gs_knn_combined.best_estimator_
y_pred_tuned_knn_combined= tuned_knn_combined.predict(X_test_combined)
y_pred_prob_tuned_knn_combined= tuned_knn_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_combined, labels= tuned_knn_combined.classes_)
print(classification_report(y_test, y_pred_tuned_knn_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_combined))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.56      0.02      0.03       912

    accuracy                           0.90      9429
   macro avg       0.73      0.51      0.49      9429
weighted avg       0.87      0.90      0.86      9429

Precision: 0.56
Accuracy: 0.9035952911231308
AUC: 0.8137179588192036


--- GaussianNB ---

In [17]:
from sklearn.naive_bayes import GaussianNB
nb_combined= GaussianNB()
nb_combined.fit(X_train_combined, y_train)
y_pred_nb_combined= nb_combined.predict(X_test_combined)
y_pred_prob_nb_combined= nb_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_combined, labels= nb_combined.classes_)
print(classification_report(y_test, y_pred_nb_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_combined))

              precision    recall  f1-score   support

           0       0.97      0.69      0.81      8517
           1       0.21      0.77      0.33       912

    accuracy                           0.70      9429
   macro avg       0.59      0.73      0.57      9429
weighted avg       0.89      0.70      0.76      9429

Precision: 0.2121120819523953
Accuracy: 0.7006045179764556
AUC: 0.8055804670329105


--- SVM ---

In [18]:
from sklearn.svm import SVC
svc_combined= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc_combined.fit(X_train_combined, y_train)
y_pred_svc_combined= svc_combined.predict(X_test_combined)
y_pred_prob_svc_combined= svc_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_combined, labels= svc_combined.classes_)
print(classification_report(y_test, y_pred_svc_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_combined))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.17      0.00      0.01       912

    accuracy                           0.90      9429
   macro avg       0.54      0.50      0.48      9429
weighted avg       0.83      0.90      0.86      9429

Precision: 0.16666666666666666
Accuracy: 0.9020044543429844
AUC: 0.696488279890168


--- Random Forest ---

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf_combined= RandomForestClassifier(random_state=15)
rf_combined.fit(X_train_combined, y_train)
y_pred_rf_combined= rf_combined.predict(X_test_combined)
y_pred_prob_rf_combined= rf_combined.predict_proba(X_test_combined)[:,1]

In [20]:
cm= confusion_matrix(y_test, y_pred_rf_combined, labels= rf_combined.classes_)
print(classification_report(y_test, y_pred_rf_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_combined))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.43      0.07      0.12       912

    accuracy                           0.90      9429
   macro avg       0.67      0.53      0.53      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.4266666666666667
Accuracy: 0.9009438964895535
AUC: 0.8223596022609065


In [21]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_combined= GridSearchCV(estimator= rf_combined, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_combined.fit(X_train_combined, y_train)
print("Best Parameters:", gs_tuned_rf_combined.best_params_)
print("Best Precision Score:", gs_tuned_rf_combined.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
Best Precision Score: 0.7052517905379709


In [22]:
tuned_rf_combined= gs_tuned_rf_combined.best_estimator_
y_pred_tuned_rf_combined= tuned_rf_combined.predict(X_test_combined)
y_pred_prob_tuned_rf_combined= tuned_rf_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_combined, labels= tuned_rf_combined.classes_)
print(classification_report(y_test, y_pred_tuned_rf_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_combined))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      8517
           1       0.58      0.04      0.07       912

    accuracy                           0.90      9429
   macro avg       0.74      0.52      0.51      9429
weighted avg       0.87      0.90      0.86      9429

Precision: 0.5789473684210527
Accuracy: 0.9042316258351893
AUC: 0.8482993378567941


--- AdaBoost ---

In [23]:
from sklearn.ensemble import AdaBoostClassifier
ada_combined= AdaBoostClassifier(random_state=15)
ada_combined.fit(X_train_combined, y_train)
y_pred_ada_combined= ada_combined.predict(X_test_combined)
y_pred_prob_ada_combined= ada_combined.predict_proba(X_test_combined)[:,1]

In [24]:
cm= confusion_matrix(y_test, y_pred_ada_combined, labels= ada_combined.classes_)
print(classification_report(y_test, y_pred_ada_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_combined))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95      8517
           1       0.53      0.16      0.25       912

    accuracy                           0.91      9429
   macro avg       0.73      0.57      0.60      9429
weighted avg       0.88      0.91      0.88      9429

Precision: 0.5347985347985348
Accuracy: 0.9052921836886202
AUC: 0.8532928338369701


In [25]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_combined= GridSearchCV(estimator= ada_combined, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_combined.fit(X_train_combined, y_train)
print("Best Parameters:", gs_ada_combined.best_params_)
print("Best Precision Score:", gs_ada_combined.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.5613090844520825


In [26]:
tuned_ada_combined= gs_ada_combined.best_estimator_
y_pred_tuned_ada_combined= tuned_ada_combined.predict(X_test_combined)
y_pred_prob_tuned_ada_combined= tuned_ada_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_combined, labels= tuned_ada_combined.classes_)
print(classification_report(y_test, y_pred_tuned_ada_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_combined))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.52      0.16      0.25       912

    accuracy                           0.90      9429
   macro avg       0.72      0.57      0.60      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.5211267605633803
Accuracy: 0.9045497931912185
AUC: 0.8519154930592889


--- GradientBoosting ---

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
grb_combined= GradientBoostingClassifier(random_state=15)
grb_combined.fit(X_train_combined, y_train)
y_pred_grb_combined= grb_combined.predict(X_test_combined)
y_pred_prob_grb_combined= grb_combined.predict_proba(X_test_combined)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_combined, labels= grb_combined.classes_)
print(classification_report(y_test, y_pred_grb_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_combined))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.53      0.12      0.20       912

    accuracy                           0.90      9429
   macro avg       0.72      0.56      0.57      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.5336538461538461
Accuracy: 0.9047619047619048
AUC: 0.8532785435321307


In [28]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_combined= GridSearchCV(estimator= grb_combined, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_combined.fit(X_train_combined, y_train)

print("Best Parameters:", gs_grb_combined.best_params_)
print("Best Precision Score:", gs_grb_combined.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.01}
Best Precision Score: 0.5933333333333334


In [29]:
tuned_grb_combined= gs_grb_combined.best_estimator_
y_pred_tuned_grb_combined= tuned_grb_combined.predict(X_test_combined)
y_pred_prob_tuned_grb_combined= tuned_grb_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_combined, labels= tuned_grb_combined.classes_)
print(classification_report(y_test, y_pred_tuned_grb_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_combined))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       1.00      0.00      0.00       912

    accuracy                           0.90      9429
   macro avg       0.95      0.50      0.48      9429
weighted avg       0.91      0.90      0.86      9429

Precision: 1.0
Accuracy: 0.9033831795524446
AUC: 0.8297489772776427


--- XGB ---

In [30]:
from xgboost import XGBClassifier
xgb_combined= XGBClassifier(random_state=15)
xgb_combined.fit(X_train_combined, y_train)
y_pred_xgb_combined= xgb_combined.predict(X_test_combined)
y_pred_prob_xgb_combined= xgb_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_combined, labels= xgb_combined.classes_)
print(classification_report(y_test, y_pred_xgb_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_combined))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.47      0.16      0.24       912

    accuracy                           0.90      9429
   macro avg       0.69      0.57      0.59      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.4670846394984326
Accuracy: 0.9010499522748966
AUC: 0.8381440807754976


In [31]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_combined= GridSearchCV(estimator= xgb_combined, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_combined.fit(X_train_combined, y_train)

print("Best Parameters:", gs_xgb_combined.best_params_)
print("Best Precision Score:", gs_xgb_combined.best_score_)

tuned_xgb_combined= gs_xgb_combined.best_estimator_
y_pred_tuned_xgb_combined= tuned_xgb_combined.predict(X_test_combined)
y_pred_prob_tuned_xgb_combined= tuned_xgb_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_combined, labels= tuned_xgb_combined.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_combined))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.01}
Best Precision Score: 0.7297619047619048
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.50      0.01      0.02       912

    accuracy                           0.90      9429
   macro avg       0.70      0.50      0.48      9429
weighted avg       0.86      0.90      0.86      9429

Precision: 0.5
Accuracy: 0.9032771237671015
AUC: 0.8395279873689154


--- LGBM ---

In [32]:
from lightgbm import LGBMClassifier
lgm_combined= LGBMClassifier(random_state=15)
lgm_combined.fit(X_train_combined, y_train)
y_pred_lgm_combined= lgm_combined.predict(X_test_combined)
y_pred_prob_lgm_combined= lgm_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_combined, labels= lgm_combined.classes_)
print(classification_report(y_test, y_pred_lgm_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_combined))

[LightGBM] [Info] Number of positive: 3649, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016206 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 506
[LightGBM] [Info] Number of data points in the train set: 37714, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.096755 -> initscore=-2.233817
[LightGBM] [Info] Start training from score -2.233817
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.50      0.11      0.18       912

    accuracy                           0.90      9429
   macro avg       0.70      0.55      0.57      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.4951923076923077
Accuracy: 0.9030650121964153
AUC: 0.8498403734327012


In [33]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_combined= GridSearchCV(estimator= lgm_combined, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_combined.fit(X_train_combined, y_train)

print("Best Parameters:", gs_lgm_combined.best_params_)
print("Best Precision Score:", gs_lgm_combined.best_score_)

tuned_lgm_combined= gs_lgm_combined.best_estimator_
y_pred_tuned_lgm_combined= tuned_lgm_combined.predict(X_test_combined)
y_pred_prob_tuned_lgm_combined= tuned_lgm_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_combined, labels= tuned_lgm_combined.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_combined))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Info] Number of positive: 3649, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015555 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 506
[LightGBM] [Info] Number of data points in the train set: 37714, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.096755 -> initscore=-2.233817
[LightGBM] [Info] Start training from score -2.233817
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 400, 'num_leaves': 31, 'reg_alpha': 0.5}
Best Precision Score: 0.573501601

--- CatBoost ---

In [34]:
from catboost import CatBoostClassifier
cat_combined= CatBoostClassifier(random_state=15)
cat_combined.fit(X_train_combined, y_train)
y_pred_cat_combined= cat_combined.predict(X_test_combined)
y_pred_prob_cat_combined= cat_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_combined, labels= cat_combined.classes_)
print(classification_report(y_test, y_pred_cat_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_combined))

Learning rate set to 0.04854
0:	learn: 0.6398277	total: 167ms	remaining: 2m 46s
1:	learn: 0.5936016	total: 184ms	remaining: 1m 31s
2:	learn: 0.5544625	total: 208ms	remaining: 1m 9s
3:	learn: 0.5202186	total: 223ms	remaining: 55.6s
4:	learn: 0.4898430	total: 239ms	remaining: 47.5s
5:	learn: 0.4665616	total: 255ms	remaining: 42.3s
6:	learn: 0.4415526	total: 272ms	remaining: 38.5s
7:	learn: 0.4214460	total: 286ms	remaining: 35.4s
8:	learn: 0.4006967	total: 301ms	remaining: 33.1s
9:	learn: 0.3865920	total: 315ms	remaining: 31.2s
10:	learn: 0.3736702	total: 329ms	remaining: 29.6s
11:	learn: 0.3594068	total: 342ms	remaining: 28.2s
12:	learn: 0.3487720	total: 357ms	remaining: 27.1s
13:	learn: 0.3390812	total: 383ms	remaining: 27s
14:	learn: 0.3312052	total: 399ms	remaining: 26.2s
15:	learn: 0.3230133	total: 417ms	remaining: 25.6s
16:	learn: 0.3159163	total: 433ms	remaining: 25s
17:	learn: 0.3103558	total: 448ms	remaining: 24.4s
18:	learn: 0.3054374	total: 462ms	remaining: 23.9s
19:	learn: 0.3

173:	learn: 0.2347349	total: 2.88s	remaining: 13.7s
174:	learn: 0.2346600	total: 2.89s	remaining: 13.6s
175:	learn: 0.2345737	total: 2.9s	remaining: 13.6s
176:	learn: 0.2344550	total: 2.92s	remaining: 13.6s
177:	learn: 0.2343652	total: 2.93s	remaining: 13.5s
178:	learn: 0.2342725	total: 2.94s	remaining: 13.5s
179:	learn: 0.2341938	total: 2.96s	remaining: 13.5s
180:	learn: 0.2341092	total: 2.97s	remaining: 13.4s
181:	learn: 0.2340053	total: 2.98s	remaining: 13.4s
182:	learn: 0.2339335	total: 3s	remaining: 13.4s
183:	learn: 0.2338541	total: 3.01s	remaining: 13.4s
184:	learn: 0.2338125	total: 3.03s	remaining: 13.3s
185:	learn: 0.2337255	total: 3.04s	remaining: 13.3s
186:	learn: 0.2336337	total: 3.06s	remaining: 13.3s
187:	learn: 0.2335441	total: 3.08s	remaining: 13.3s
188:	learn: 0.2334551	total: 3.09s	remaining: 13.3s
189:	learn: 0.2333578	total: 3.11s	remaining: 13.3s
190:	learn: 0.2332940	total: 3.12s	remaining: 13.2s
191:	learn: 0.2332285	total: 3.14s	remaining: 13.2s
192:	learn: 0.23

338:	learn: 0.2224962	total: 5.09s	remaining: 9.93s
339:	learn: 0.2224208	total: 5.1s	remaining: 9.9s
340:	learn: 0.2223530	total: 5.11s	remaining: 9.88s
341:	learn: 0.2222903	total: 5.13s	remaining: 9.86s
342:	learn: 0.2222421	total: 5.14s	remaining: 9.84s
343:	learn: 0.2221595	total: 5.15s	remaining: 9.82s
344:	learn: 0.2220864	total: 5.16s	remaining: 9.8s
345:	learn: 0.2220012	total: 5.17s	remaining: 9.78s
346:	learn: 0.2219502	total: 5.18s	remaining: 9.76s
347:	learn: 0.2218880	total: 5.2s	remaining: 9.74s
348:	learn: 0.2218309	total: 5.21s	remaining: 9.71s
349:	learn: 0.2217777	total: 5.22s	remaining: 9.69s
350:	learn: 0.2217051	total: 5.23s	remaining: 9.68s
351:	learn: 0.2216653	total: 5.24s	remaining: 9.65s
352:	learn: 0.2215949	total: 5.26s	remaining: 9.64s
353:	learn: 0.2215039	total: 5.27s	remaining: 9.63s
354:	learn: 0.2214541	total: 5.29s	remaining: 9.61s
355:	learn: 0.2214135	total: 5.3s	remaining: 9.6s
356:	learn: 0.2213594	total: 5.32s	remaining: 9.58s
357:	learn: 0.2212

510:	learn: 0.2123150	total: 7.46s	remaining: 7.14s
511:	learn: 0.2122521	total: 7.47s	remaining: 7.12s
512:	learn: 0.2122062	total: 7.48s	remaining: 7.1s
513:	learn: 0.2121544	total: 7.5s	remaining: 7.09s
514:	learn: 0.2120862	total: 7.51s	remaining: 7.07s
515:	learn: 0.2120193	total: 7.53s	remaining: 7.06s
516:	learn: 0.2119960	total: 7.54s	remaining: 7.04s
517:	learn: 0.2119531	total: 7.57s	remaining: 7.04s
518:	learn: 0.2118754	total: 7.58s	remaining: 7.02s
519:	learn: 0.2118148	total: 7.59s	remaining: 7.01s
520:	learn: 0.2117755	total: 7.6s	remaining: 6.99s
521:	learn: 0.2116997	total: 7.61s	remaining: 6.97s
522:	learn: 0.2116268	total: 7.63s	remaining: 6.96s
523:	learn: 0.2115842	total: 7.64s	remaining: 6.94s
524:	learn: 0.2115504	total: 7.66s	remaining: 6.93s
525:	learn: 0.2114689	total: 7.67s	remaining: 6.91s
526:	learn: 0.2114304	total: 7.68s	remaining: 6.9s
527:	learn: 0.2113839	total: 7.7s	remaining: 6.88s
528:	learn: 0.2113195	total: 7.71s	remaining: 6.86s
529:	learn: 0.211

671:	learn: 0.2041065	total: 9.45s	remaining: 4.61s
672:	learn: 0.2040752	total: 9.46s	remaining: 4.6s
673:	learn: 0.2040449	total: 9.48s	remaining: 4.58s
674:	learn: 0.2039755	total: 9.49s	remaining: 4.57s
675:	learn: 0.2039138	total: 9.5s	remaining: 4.55s
676:	learn: 0.2038333	total: 9.51s	remaining: 4.54s
677:	learn: 0.2037956	total: 9.52s	remaining: 4.52s
678:	learn: 0.2037541	total: 9.53s	remaining: 4.51s
679:	learn: 0.2036885	total: 9.54s	remaining: 4.49s
680:	learn: 0.2036372	total: 9.56s	remaining: 4.48s
681:	learn: 0.2035996	total: 9.57s	remaining: 4.46s
682:	learn: 0.2035535	total: 9.58s	remaining: 4.45s
683:	learn: 0.2035181	total: 9.59s	remaining: 4.43s
684:	learn: 0.2035034	total: 9.6s	remaining: 4.42s
685:	learn: 0.2034769	total: 9.61s	remaining: 4.4s
686:	learn: 0.2034300	total: 9.62s	remaining: 4.38s
687:	learn: 0.2034030	total: 9.64s	remaining: 4.37s
688:	learn: 0.2033430	total: 9.66s	remaining: 4.36s
689:	learn: 0.2032974	total: 9.67s	remaining: 4.34s
690:	learn: 0.20

836:	learn: 0.1966300	total: 11.4s	remaining: 2.23s
837:	learn: 0.1966082	total: 11.5s	remaining: 2.21s
838:	learn: 0.1965822	total: 11.5s	remaining: 2.2s
839:	learn: 0.1965556	total: 11.5s	remaining: 2.19s
840:	learn: 0.1965165	total: 11.5s	remaining: 2.17s
841:	learn: 0.1964623	total: 11.5s	remaining: 2.16s
842:	learn: 0.1963986	total: 11.5s	remaining: 2.15s
843:	learn: 0.1963187	total: 11.5s	remaining: 2.13s
844:	learn: 0.1962740	total: 11.5s	remaining: 2.12s
845:	learn: 0.1962222	total: 11.6s	remaining: 2.1s
846:	learn: 0.1961742	total: 11.6s	remaining: 2.09s
847:	learn: 0.1961391	total: 11.6s	remaining: 2.07s
848:	learn: 0.1961020	total: 11.6s	remaining: 2.06s
849:	learn: 0.1960748	total: 11.6s	remaining: 2.05s
850:	learn: 0.1960338	total: 11.6s	remaining: 2.03s
851:	learn: 0.1959974	total: 11.6s	remaining: 2.02s
852:	learn: 0.1959778	total: 11.6s	remaining: 2s
853:	learn: 0.1959421	total: 11.6s	remaining: 1.99s
854:	learn: 0.1959214	total: 11.7s	remaining: 1.98s
855:	learn: 0.195

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.51      0.14      0.22       912

    accuracy                           0.90      9429
   macro avg       0.71      0.56      0.58      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.5100401606425703
Accuracy: 0.903807402693817
AUC: 0.8480085752128353


In [35]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_combined= GridSearchCV(estimator= cat_combined, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_combined.fit(X_train_combined, y_train)

print("Best Parameters:", gs_cat_combined.best_params_)
print("Best Precision Score:", gs_cat_combined.best_score_)

tuned_cat_combined= gs_cat_combined.best_estimator_
y_pred_tuned_cat_combined= tuned_cat_combined.predict(X_test_combined)
y_pred_prob_tuned_cat_combined= tuned_cat_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_combined, labels= tuned_cat_combined.classes_)
print(classification_report(y_test, y_pred_tuned_cat_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_combined))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.5910857	total: 17.1ms	remaining: 154ms
1:	learn: 0.5141547	total: 32.8ms	remaining: 131ms
2:	learn: 0.4598812	total: 47.5ms	remaining: 111ms
3:	learn: 0.4155299	total: 59.3ms	remaining: 89ms
4:	learn: 0.3830648	total: 73.5ms	remaining: 73.5ms
5:	learn: 0.3568865	total: 88ms	remaining: 58.7ms
6:	learn: 0.3400753	total: 99.7ms	remaining: 42.7ms
7:	learn: 0.3224750	total: 112ms	remaining: 28ms
8:	learn: 0.3084385	total: 124ms	remaining: 13.8ms
9:	learn: 0.2997350	total: 137ms	remaining: 0us
Best Parameters: {'depth': 5, 'iterations': 10, 'learning_rate': 0.1}
Best Precision Score: 0.8683982683982684
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       1.00      0.00      0.00       912

    accuracy                           0.90      9429
   macro avg       0.95      0.50      0.48      9429
weighted avg       0.91      0.90      0.86      9

--- Visualisation ---

In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_combined, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_combined, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_combined, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_combined, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_combined, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_combined, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_combined, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_combined, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_combined, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_combined, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_combined, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_combined, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_combined, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_combined, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_combined, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_combined, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_combined, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_combined, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_combined, zero_division= 0)
                  }

combined_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
combined_precision= combined_precision.sort_values(by= 'Precision Score', ascending=False)
print(combined_precision)

                                    Model  Precision Score
18              Tuned CatBoost Precision:         1.000000
12      Tuned GradientBoosting Precision:         1.000000
8          Tuned Random Forest Precision:         0.578947
0          Logistic Regression Precision:         0.560185
4   Tuned KNeighborsClassifier Precision:         0.560000
9                     AdaBoost Precision:         0.534799
11            GradientBoosting Precision:         0.533654
16                  Tuned LGBM Precision:         0.526316
10              Tuned AdaBoost Precision:         0.521127
17                    CatBoost Precision:         0.510040
14                   Tuned XGB Precision:         0.500000
15                        LGBM Precision:         0.495192
13                         XGB Precision:         0.467085
2          Tuned Decision Tree Precision:         0.430556
7                Random Forest Precision:         0.426667
3         KNeighborsClassifier Precision:         0.3642

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_combined),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_combined),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_combined),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_combined),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_combined),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_combined),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_combined),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_combined),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_combined),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_combined),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_combined),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_combined),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_combined),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_combined),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_combined),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_combined),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_combined),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_combined),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_combined)
                  }

combined_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
combined_accuracy= combined_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(combined_accuracy)

                                   Model  Accuracy Score
0          Logistic Regression Accuracy:        0.906035
9                     AdaBoost Accuracy:        0.905292
11            GradientBoosting Accuracy:        0.904762
10              Tuned AdaBoost Accuracy:        0.904550
16                  Tuned LGBM Accuracy:        0.904550
8          Tuned Random Forest Accuracy:        0.904232
17                    CatBoost Accuracy:        0.903807
4   Tuned KNeighborsClassifier Accuracy:        0.903595
18              Tuned CatBoost Accuracy:        0.903489
12      Tuned GradientBoosting Accuracy:        0.903383
14                   Tuned XGB Accuracy:        0.903277
15                        LGBM Accuracy:        0.903065
6                          SVM Accuracy:        0.902004
2          Tuned Decision Tree Accuracy:        0.901156
13                         XGB Accuracy:        0.901050
7                Random Forest Accuracy:        0.900944
3         KNeighborsClassifier 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_combined),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_combined),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_combined),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_combined),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_combined),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_combined),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_combined),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_combined),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_combined),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_combined),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_combined),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_combined),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_combined),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_combined),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_combined),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_combined),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_combined),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_combined),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_combined)
                  }

combined_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
combined_auc= combined_auc.sort_values(by= 'AUC Score', ascending=False)
print(combined_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.854134
9                     AdaBoost AUC:   0.853293
11            GradientBoosting AUC:   0.853279
10              Tuned AdaBoost AUC:   0.851915
15                        LGBM AUC:   0.849840
16                  Tuned LGBM AUC:   0.849110
8          Tuned Random Forest AUC:   0.848299
17                    CatBoost AUC:   0.848009
14                   Tuned XGB AUC:   0.839528
13                         XGB AUC:   0.838144
12      Tuned GradientBoosting AUC:   0.829749
18              Tuned CatBoost AUC:   0.828227
7                Random Forest AUC:   0.822360
4   Tuned KNeighborsClassifier AUC:   0.813718
5                   GaussianNB AUC:   0.805580
2          Tuned Decision Tree AUC:   0.802002
3         KNeighborsClassifier AUC:   0.702689
6                          SVM AUC:   0.696488
1                Decision Tree AUC:   0.588169
